# IMPORT

In [ ]:
from tensorflow import keras
from keras import layers, Sequential
import numpy as np
import pandas as pd
from keras import backend as K
from keras.utils import  to_categorical
from sklearn.model_selection import KFold
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler


In [ ]:
!wget -nc -O dataset.zip https://www.dropbox.com/scl/fo/9ppie6m2voe9s68ato4iv/h?rlkey=m23k38d3xka34l9kdytpxm0ex&dl=0
!unzip -n -d dataset dataset.zip

File ‘dataset.zip’ already there; not retrieving.
Archive:  dataset.zip
mapname:  conversion of  failed


# IMPORT DATA

In [ ]:
X = pd.read_csv("selected_features.csv")
data = loadmat('dataset/Project_data.mat')


In [ ]:
X = X.to_numpy()


In [ ]:
y = data['TrainLabels'].T
y[ y == -1] = 0

# VARIABLES

In [ ]:
input_shape = 50 #TODO calculate using features size
epochs = 30

In [ ]:
X.shape

(550, 50)

In [ ]:
def normalize(feature):
    scaler = MinMaxScaler()
    normalized_feature = scaler.fit_transform(feature)
    return normalized_feature

In [ ]:
X=normalize(X)

# DEFINITIONS

In [ ]:
class RBFLayer(layers.Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

# MODELS

In [ ]:
model_mlp_sigmoid = Sequential([
    layers.InputLayer(input_shape=(input_shape,)),
    layers.Dense(units = 32 , activation = "relu"),
    layers.Dense(units = 64 , activation = "relu"),
    layers.Dense(units = 128 , activation = "relu"),
    layers.Dense(units = 64 , activation = "relu"),
    layers.Dense(units = 32 , activation = "relu"),
    layers.Dense(units = 1 , activation = "sigmoid"),
])
model_mlp_softmax = Sequential([
    layers.InputLayer(input_shape=(input_shape,)),
    layers.Dense(units = 32 , activation = "relu"),
    layers.Dense(units = 64 , activation = "relu"),
    layers.Dense(units = 128 , activation = "relu"),
    layers.Dense(units = 64 , activation = "relu"),
    layers.Dense(units = 32 , activation = "relu"),
    layers.Dense(units = 2 , activation = "softmax"),
])
model_mlp_sigmoid.compile(optimizer = "adam" , loss = "binary_crossentropy" , metrics = ["accuracy", "binary_accuracy"])
model_mlp_softmax.compile(optimizer = "adam" , loss = "categorical_crossentropy" , metrics = ["accuracy", "categorical_accuracy"])

In [ ]:
model_rbf_sigmoid = keras.Sequential([
    layers.InputLayer(input_shape=(input_shape,)),
    layers.experimental.RandomFourierFeatures(output_dim=550,scale=10.,kernel_initializer='gaussian'),
    layers.Dense(units=1, activation='sigmoid'),
])
model_rbf_softmax = keras.Sequential([
    layers.InputLayer(input_shape=(input_shape,)),
    layers.experimental.RandomFourierFeatures(output_dim=550,scale=10.,kernel_initializer='gaussian'),
    layers.Dense(units=2, activation='softmax'),
])
model_rbf_sigmoid.compile(optimizer='adam',loss='binary_crossentropy',metrics = ["accuracy", "binary_accuracy"])
model_rbf_softmax.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ["accuracy", "categorical_accuracy"])

In [ ]:
model_2_rbf_sigmoid = keras.Sequential([
    layers.InputLayer(input_shape=(input_shape,)),
    RBFLayer(550, 0.5),
    layers.Dense(units=1, activation='sigmoid'),
])
model_2_rbf_softmax = keras.Sequential([
    layers.InputLayer(input_shape=(input_shape,)),
    RBFLayer(550, 0.5),
    layers.Dense(units=2, activation='softmax'),
])
model_2_rbf_sigmoid.compile(optimizer='adam',loss='binary_crossentropy',metrics = ["accuracy", "binary_accuracy"])
model_2_rbf_softmax.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ["accuracy", "categorical_accuracy"])

In [ ]:
kf = KFold(n_splits=5)
# kf.get_n_splits(X)
histories =  []
histories_mlp_sigmoid =[]
histories_mlp_softmax =[]
histories_rbf_1_sigmoid =[]
histories_rbf_1_softmax =[]
histories_rbf_2_sigmoid =[]
fold_accuracies = []
kf.get_n_splits(X)

5

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print(train_index)
    X_train = X[train_index , :]
    y_train = y[train_index , :]
    X_valid = X[test_index , :]
    y_valid = y[test_index , :]
    y_categorized = to_categorical(y)
    y_train_categorized = y_categorized[train_index , :]
    y_valid_categorized = y_categorized[test_index , :]
    #TODO after import Data open codes
    print(f"##################### Fold :{i} started #####################")
    print("MLP sigmoid started")
    history_mlp_sigmoid = model_mlp_sigmoid.fit(X_train , y_train , epochs= epochs , validation_data = (X_valid, y_valid), verbose = 0)
    print("MLP sigmoid finished")
    histories_mlp_sigmoid.append(history_mlp_sigmoid)
    print("MLP softmax started")
    history_mlp_softmax = model_mlp_softmax.fit(X_train , y_train_categorized , epochs= epochs , validation_data = (X_valid, y_valid_categorized), verbose = 0)
    print("MLP softmax finished")
    histories_mlp_softmax.append(history_mlp_softmax)
    #TODO after import Data open codes
    print("RBF keras builtin sigmoid started")
    history_rbf_sigmoid = model_rbf_sigmoid.fit(X_train , y_train , epochs= epochs , validation_data = (X_valid, y_valid), verbose = 0)
    print("RBF keras builtin sigmoid finished")
    histories_rbf_1_sigmoid.append(history_rbf_sigmoid)
    print("RBF keras builtin softmax started")
    history_rbf_softmax = model_rbf_softmax.fit(X_train , y_train_categorized , epochs= epochs , validation_data = (X_valid, y_valid_categorized), verbose = 0)
    print("RBF keras builtin softmax finished")
    histories_rbf_1_softmax.append(history_rbf_softmax)
    # TODO after import Data open codes
    print("RBF custom class sigmoid started")
    history_2_rbf_sigmoid = model_2_rbf_sigmoid.fit(X_train , y_train , epochs= epochs , validation_data = (X_valid, y_valid), verbose = 0)
    print("RBF custom class sigmoid finished")
    histories_rbf_2_sigmoid.append(history_2_rbf_sigmoid)

    # histories.append({
    #     "mlp_sigomoid" :history_mlp_sigmoid ,
    #     "mlp_softmax" :history_mlp_softmax ,
    #     "rbf_1_sigomoid" :history_rbf_sigmoid ,
    #     "rbf_1_softmax" :history_rbf_softmax ,
    #     "rbf_2_sigomoid" :history_2_rbf_sigmoid,
    #     })
    # val_predictions = (model_mlp_sigmoid.predict(X_valid) > 0.5).astype(int)
    # accuracy = accuracy_score(y_valid, val_predictions)
    # fold_accuracies.append(accuracy)
    # histories_rbf_2_sigmoid.append(history_2_rbf_sigmoid)
    # print("RBF custom class softmax started")
    # history_2_rbf_2 = model_2_rbf_2_channels.fit(X_train , y_train_categorized , epochs= 100 , validation_data = (X_valid, y_valid_categorized), verbose = 0)
    # print("RBF custom class softmax finished")
# average_accuracy = np.mean(fold_accuracies)
# print(average_accuracy)


##################### Fold :0 started #####################
MLP sigmoid started
MLP sigmoid finished
MLP softmax started
MLP softmax finished
RBF keras builtin sigmoid started
RBF keras builtin sigmoid finished
RBF keras builtin softmax started
RBF keras builtin softmax finished
RBF custom class sigmoid started
RBF custom class sigmoid finished
##################### Fold :1 started #####################
MLP sigmoid started
MLP sigmoid finished
MLP softmax started
MLP softmax finished
RBF keras builtin sigmoid started
RBF keras builtin sigmoid finished
RBF keras builtin softmax started
RBF keras builtin softmax finished
RBF custom class sigmoid started
RBF custom class sigmoid finished
##################### Fold :2 started #####################
MLP sigmoid started
MLP sigmoid finished
MLP softmax started
MLP softmax finished
RBF keras builtin sigmoid started
RBF keras builtin sigmoid finished
RBF keras builtin softmax started
RBF keras builtin softmax finished
RBF custom class sigmoid s

In [ ]:
print("############################### FOLDS Accuracy ###############################")
print("--------SIGMOID MLP")
for m in histories_mlp_sigmoid:
    print(max(m.history["accuracy"]))
print("--------SOFTMAX MLP")
for m in histories_mlp_softmax:
    print(max(m.history["accuracy"]))
print("--------SIGMOID RBF (Builtin Class)")
for m in histories_rbf_1_sigmoid:
    print(max(m.history["accuracy"]))
print("--------SOFTMAX RBF (Builtin Class)")
for m in histories_rbf_1_softmax:
    print(max(m.history["accuracy"]))
print("--------SIGMOID RBF (Custom Class)")
for m in histories_rbf_2_sigmoid:
    print(max(m.history["accuracy"]))


############################### FOLDS Accuracy ###############################
--------SIGMOID MLP
0.9159091114997864
0.9954545497894287
0.9977272748947144
0.9977272748947144
1.0
--------SOFTMAX MLP
0.9386363625526428
1.0
1.0
1.0
1.0
--------SIGMOID RBF (Builtin Class)
0.6727272868156433
0.7113636136054993
0.7272727489471436
0.699999988079071
0.7295454740524292
--------SOFTMAX RBF (Builtin Class)
0.6727272868156433
0.7159090638160706
0.7363636493682861
0.7090908885002136
0.7363636493682861
--------SIGMOID RBF (Custom Class)
0.6704545617103577
0.7272727489471436
0.7477272748947144
0.7272727489471436
0.75


In [ ]:
print("############################### Average Accuracy ###############################")
print("--------SIGMOID MLP")
print(sum([ max(m.history["accuracy"]) for m in  histories_mlp_sigmoid]) / 5)
print("--------SOFTMAX MLP")
print(sum([ max(m.history["accuracy"]) for m in  histories_mlp_softmax]) / 5)
print("--------SIGMOID RBF (Builtin Class)")
print(sum([ max(m.history["accuracy"]) for m in  histories_rbf_1_sigmoid]) / 5)
print("--------SOFTMAX RBF (Builtin Class)")
print(sum([ max(m.history["accuracy"]) for m in  histories_rbf_1_softmax]) / 5)
print("--------SIGMOID RBF (Custom Class)")
print(sum([ max(m.history["accuracy"]) for m in  histories_rbf_2_sigmoid]) / 5)


############################### Average Accuracy ###############################
--------SIGMOID MLP
0.9813636422157288
--------SOFTMAX MLP
0.9877272725105286
--------SIGMOID RBF (Builtin Class)
0.7081818222999573
--------SOFTMAX RBF (Builtin Class)
0.7140909075737
--------SIGMOID RBF (Custom Class)
0.7245454668998719
